# Estimating the impacts of land use and climate change on Scotland's water-related ecosystem services

## 1. Background

Under the requirements of the EU [Water Framework Directive (WFD)](https://en.wikipedia.org/wiki/Water_Framework_Directive), the Scottish Environmental Protection Agency (SEPA) must undertake a process of [River Basin Management Planning (RBMP)](http://www.sepa.org.uk/environment/water/river-basin-management-planning/), with the aim of improving water quality while also safeguarding a broad range of water-related ecosystem services (ES). As part of the second cycle of RBMP, a high-level screening methodology has been developed by SEPA and the James Hutton Institute to assess the risks posed by future land use and climate change to key Scottish ES. A [project summary](http://www.crew.ac.uk/publications/estimating-impacts-land-use-and-climate-change-scotland%E2%80%99s-water-related-ecosystem-servi) and [full report](http://www.crew.ac.uk/publications/estimating-impacts-land-use-and-climate-change-scotland%E2%80%99s-water-related-ecosystem-ser-0) are available via Scotland's Centre of Expertise for Water (CREW); this repository provides the code used in the analysis.

## 2. Key datasets

Spatial datasets describing the presence or absence of different ES in Scotland's WFD waterbody catchments are available from [SEWeb](http://www.sepa.org.uk/data-visualisation/benefits-of-the-water-environment/). This project focused on a sub-set of these ES relevant to the second cycle of RBMP and for which good quality data are available ([Table 1](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Tables/ES_List_Simple.csv)).

Gridded climate simulations for rainfall and potential evapotranspiration (PET) were taken from the [Future Flows project](https://data.gov.uk/dataset/future-flows-climate-data1), providing 11 nationwide simulations from 1951 to 2098. These simulations are all based on a single climate model ([HadRM3](http://ukclimateprojections.metoffice.gov.uk/23234)) under the assumption of the medium (A1B) emissions scenario. They therefore give an indication of the range of climate change uncertainty, subject to these assumptions.

A baseline land use dataset was created from the [Land Cover Map of Scotland 1988](https://data.gov.uk/dataset/land-cover-scotland-lcs-1988). A single future land use simulation for the 2050s was also developed using the [Scottish Land Use Strategy](http://www.gov.scot/Topics/Environment/Countryside/Landusestrategy), the [Land Capability for Forestry map](http://www.soils-scotland.gov.uk/data/soil-lcf) and the [Land Capability for Agriculture 2050 dataset](http://www.int-res.com/abstracts/cr/v37/n1/p43-57/). All land use simulations used the simplified land cover categories given in [Table 2](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Tables/Land_Use_Classes.csv) (see the [main report](http://www.crew.ac.uk/publications/estimating-impacts-land-use-and-climate-change-scotland%E2%80%99s-water-related-ecosystem-ser-0) for full details).

The gridded datasets for the baseline and future climate simulations were stored as numpy arrays within an [HDF5](https://www.hdfgroup.org/HDF5/) datastore.

## 3. Risk assessment framework

Process-based modelling of 12 ES at national scale was beyond the scope of this project. Instead, a qualitative approach was taken, underpinned by expert elicitation and the vulnerability assessment framework defined by the [IPCC in their 3rd assessment report](http://onlinelibrary.wiley.com/doi/10.1002/joc.775/abstract;jsessionid=46CA147CF585DFFCED0E1BA7A4359781.f03t03). All model outputs were translated onto the following **qualitative scale**:

| Score |   Change in ES provision  |
|:-----:|:--------------:|
|   +2  | Large increase |
|   +1  | Small increase |
|   0   |    No change   |
|   -1  | Small decrease |
|   -2  | Large decrease |

Change scores were assigned following a **workshop** where two expert groups were asked to complete a variety of exercises to encapsulate their opinions regarding the vulnerability of each ES to future change. Results from these exercies were used to generate **transition matrices** for each group, which were then used to translate changes in rainfall, PET and land use into changes in ES provision. Example matrices from **group 1** are provided [here](https://github.com/JamesSample/ecosystem_services_impacts/tree/master/Matrices).

## 4. Model output

For each ES and each of three future scenarios (land use change only, climate change only and land use & climate change combined), the model produces pairs of maps like the one below. The left-hand panel shows the **median** modelled change for each water body where the ES is known to be present; the right-hand panel shows **confidence scores**, which reflect both climate change uncertainty and the diversity of expert opinion expressed at the workshop. 

![Model output](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Images/ES_04_Comb.png?raw=true)

## 5. Code structure

The [scripts provided](https://github.com/JamesSample/ecosystem_services_impacts/tree/master/Code) read gridded data from the HDF5 store and combine it with reclassification matrices (such as the examples [here](https://github.com/JamesSample/ecosystem_services_impacts/tree/master/Matrices)) to produce maps such as the ones shown above. The three scripts are run sequentially and perform the following functions:

1. **[01_es_lu_cc.py](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Code/01_es_lu_cc.py)**. Reads the raw climate and land use data and combines it with the reclassification matrices from the workshop. Produces maps on a 1 km grid where each grid cell has an associated change score. <br><br>

2. **[02_wb_zonal_stats.py](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Code/02_wb_zonal_stats.py)**. Spatially aggregates the 1 km gridded output from step 1 by calculating the **median** score within each WFD waterbody catchment. Also calculates the **range** of scores (maximum minus minimum) for each of the three future scenarios. <br><br>

3. **[03_map_outputs.py](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Code/03_map_outputs.py)**. Uses [ArcPy](http://help.arcgis.com/en/arcgisdesktop/10.0/help/index.html#//000v000000v7000000.htm) to control the layout and cartographic display of the output from step 2. The final output is a set of 36 pairs of maps, in both PDF and PNG format. This script makes use of the ArcMap template document and layer symbology files located [here](https://github.com/JamesSample/ecosystem_services_impacts/tree/master/ArcGIS). Note this script must be run using **32-bit Python** to avoid problems with the ArcPy DLL.

The first two scripts perform all the data processing and are underpinned by a variety of open source Python modules. The third script uses the proprietary ArcPy module, so users will need a licence for [ArcGIS Desktop](http://www.esri.com/software/arcgis/arcgis-for-desktop) in order to run it.

### 5.1. 01_es_lu_cc.py

The first script requires the user to specify the following information:

* **ff_h5_path**. Path to the HDF5 file containing the Future Flows data. <br><br>

* **ro_path**. Path to the runoff matrix for the group under consideration e.g. [here](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Matrices/Runoff_Impacts_Grp1.csv). <br><br>

* **ro_matrix_15**. Reclassification matrix for rainfall and PET e.g. [here](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Matrices/Runoff_Matrix_15pct.csv). <br><br>

* **base_path** and **fut_path**. Paths to ASCII grids of the baseline and future land use simulations. <br><br>

* **lu_classes_path**. List of land use classes under consideration e.g. [here](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Tables/Land_Use_Classes.csv). <br><br>

* **lu_matrices_path**. Path to Excel file containing land use transition matrices based on workshop outputs e.g. [here](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Matrices/Land_Use_Matrices_Grp1.xlsx). <br><br>

* **lucc_matrices_path**. Path to Excel file containing combined land-use-and-climate-change transition matrices based on workshop outputs e.g. [here](https://github.com/JamesSample/ecosystem_services_impacts/blob/master/Matrices/Climate_And_Land_Use_Matrices_Grp1.xlsx). <br><br>

* **out_pdf_fold**. Folder in which to store PDF maps of 1 km gridded outputs (i.e. before catchment level aggregation). <br><br>

* **out_array_fold**. Folder in which to store GeoTiffs of 1 km gridded outputs (i.e. before catchment level aggregation). <br><br>

* **base_st_yr** and **base_end_yr**. Start and end point for the baseline period of interest (minimum 1951; maximum 2098). <br><br>

* **fut_st_yr** and **fut_end_yr**. Start and end point for the future period of interest (minimum 1951; maximum 2098). <br><br>

* **models**. List of FF/HadRM3 models to consider. These are specified using the code given in the *"RCM run ID"* column of the table on [this website](http://catalogue.ceda.ac.uk/uuid/f9c7d70b0e6d4c31c067e75ab67cb3cc#tab_details). It is possible to process data for all 11 models at once.

Outputs from the first script are the inputs to the second, and so on for the third. For the most part, the required inputs are self-explanatory.

### 5.2. 02_wb_zonal_stats.py

For the second script, the user must specifiy output folders from the first script, plus the following information:

* **zones_path**. Path to a shapefile containing the *total upstream area* for each waterbody of interest. These polygons are sometimes referred to as "nested catchments" and they can overlap. <br><br>

* **zones_field**. A field within `zones_path` containing a unique identifier for each waterbody. <br><br>

* **sepa_wbs_path**. A path to a shapefile containing the **non-overlapping** waterbodies of interest (sometimes referred to as "inter-catchments"). 

### 5.3. 03_map_outputs.py

For the third script, the user must specify output folders from script 2, plus paths to the ArcGIS layer files and ArcMap document template, as provided [here](https://github.com/JamesSample/ecosystem_services_impacts/tree/master/ArcGIS).